In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
import re
from datasketch import MinHash,MinHashLSH
from concurrent.futures import ThreadPoolExecutor

In [2]:
df =pd.read_csv("tweets_2022_abril_junio.csv")
df

,id,created_at,screen_name,text,favorite_count,retweet_count
0,1512186166438637582,2022-04-07 21:50:51 UTC,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...,0,0
1,1512186202367045642,2022-04-07 21:51:00 UTC,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...,0,0
2,1512186287284924418,2022-04-07 21:51:20 UTC,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...,0,0
3,1512186335754301446,2022-04-07 21:51:32 UTC,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...,0,0
4,1512186407841767424,2022-04-07 21:51:49 UTC,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...,0,0
...,...,...,...,...,...,...
4594975,1526652300709679104,2022-05-17 19:54:06 UTC,Alebarrera74,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...,0,0
4594976,1526641118460334080,2022-05-17 19:09:40 UTC,gigita29bq,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...,0,0
4594977,1526738292011462657,2022-05-18 01:35:48 UTC,Elizabe81480339,RT @Gonz1Gorjeperez: @tere_marinovic https://t...,0,0
4594978,1526855280151056386,2022-05-18 09:20:41 UTC,CastilloNafla,RT @Gonz1Gorjeperez: @tere_marinovic https://t...,0,0


In [3]:
subconjunto = df.copy()
s = 0.4
k = 4

## Limpieza

In [4]:
spanish_stopwords = stopwords.words('spanish')

In [5]:
frase = "RT @RudyPetrikowski: ⚠️ Ayúdenme por favor a conseguir la información de las normas o artículos que la CC ha rechazado.\n\nEsto es muy import…"
def eliminar_stopwords(frase):
    palabras = frase.split(" ")
    retorno = []
    for palabra in palabras:
        if not palabra in spanish_stopwords:
            retorno.append(palabra)
    return " ".join(retorno)
def procesar_frase(frase):
    if (frase.startswith("RT")):
        frase = ' '.join(frase.split(": ")[1:])
    
    # sin hashatgs
    frase = re.sub(r"#\w+", "",frase)
    # sin url
    frase = re.sub(r"https\S+", "",frase)
    # sin emojis
    patron_emoji = re.compile("[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U00002600-\U000027BF]|[\U0001F900-\U0001F9FF]|[\U0001F1E0-\U0001F1FF]|[\U0001F1F2-\U0001F1F4]|[\U0001F1E6-\U0001F1FF]")
    frase = re.sub(patron_emoji, '', frase)
    # sin espacios
    frase = re.sub(r"\s+", " ",frase)
    # eliminar tildes
    frase = frase.replace("á","a")
    frase = frase.replace("é","e")
    frase = frase.replace("í","i")
    frase = frase.replace("ó","o")
    frase = frase.replace("ú","u")

    frase = eliminar_stopwords(frase)
    
    # minusculas
    frase = frase.lower()
    frase = frase.strip()
    return frase

procesar_frase(frase)

'️ ayudenme favor conseguir informacion normas articulos cc rechazado. esto import…'

In [6]:
subconjunto = subconjunto.drop_duplicates(["screen_name","text"]).reset_index(drop=True)
subconjunto = subconjunto[["id","screen_name","text"]]
subconjunto

,id,screen_name,text
0,1512186166438637582,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,1512186202367045642,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,1512186287284924418,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,1512186335754301446,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,1512186407841767424,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...
...,...,...,...
4549967,1526652300709679104,Alebarrera74,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...
4549968,1526641118460334080,gigita29bq,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...
4549969,1526738292011462657,Elizabe81480339,RT @Gonz1Gorjeperez: @tere_marinovic https://t...
4549970,1526855280151056386,CastilloNafla,RT @Gonz1Gorjeperez: @tere_marinovic https://t...


In [7]:
subconjunto["text"] = subconjunto["text"].apply(procesar_frase)

In [8]:
def shingle(frase , tamaño_ventana):
    lista = []
    for i in range(len(frase) - tamaño_ventana +1):
        shingle = ' '.join(frase[i:i+tamaño_ventana])
        lista.append(shingle)
    return ','.join(lista)
subconjunto["text"] = subconjunto["text"].str.split().apply(lambda x: np.array(x)).apply(lambda x: shingle(x,k))

In [9]:
subconjunto = subconjunto[subconjunto['text'].apply(lambda x: len(x.split()) > 0)]

In [10]:
subconjunto = subconjunto.reset_index(drop=True)

### Tweeets-Usuario

In [11]:
agrupado = subconjunto.copy()
agrupado = agrupado.reset_index()
agrupado = agrupado.groupby('screen_name')['index'].agg(list)
agrupado = agrupado.rename('indices_en_lista')
agrupado = agrupado.reset_index().rename(columns={'index': 'indices_en_lista'})
agrupado

,screen_name,indices_en_lista
0,0000000000000JL,"[429713, 727883, 805463, 2559655, 2589505, 304..."
1,00001009082AA,[2946888]
2,0001Andino,[3798622]
3,00132MC,[2986937]
4,001Carlao,[56494]
...,...,...
198136,zzeendo,"[620265, 670342, 1947617, 3029166, 3808204]"
198137,zzmatia1,"[389525, 578074, 815492, 1026564, 1287144, 155..."
198138,zzonck,"[2280, 258787, 265477, 313297, 363429, 1060009..."
198139,zztoprock1,"[80919, 83996, 296619, 403932, 476107, 654201,..."


In [12]:
def m_caracteristica(subconjun):
    tweets = subconjun
    shingles = tweets['text'].apply(lambda x: np.array(x.split(',')))
    mlb = MultiLabelBinarizer(sparse_output=True)
    return mlb.fit_transform(shingles)

In [13]:
matrix = m_caracteristica(subconjunto)
matrix


<4019415x5806559 sparse matrix of type '<class 'numpy.int32'>'
	with 35557046 stored elements in Compressed Sparse Row format>

In [14]:
n_pm= 10

def minhash_vec(sparce_vector):
    minhash = MinHash(n_pm)
    for i in sparce_vector.indices:
        minhash.update(str(i).encode("utf8"))
    return minhash

In [15]:
thrld = 0.4
lsh = MinHashLSH(thrld,n_pm)
list_hash = []
for i in range(matrix.shape[0]):
    list_hash.append((i,minhash_vec(matrix[i])))

with lsh.insertion_session() as session:
    for i , minhash in list_hash:
        session.insert(i, minhash)

In [16]:
len(list_hash)

4019415

### Asociamos los tweets a sus buckets similares

In [17]:
def process_tweet(id_tweet):
        m = minhash_vec(matrix[id_tweet])
        aprox = lsh.query(m)
        return aprox
def transformar_lista(lisa_id_tweet):
    similares = set()

    with ThreadPoolExecutor() as executor:
        results = executor.map(process_tweet, lisa_id_tweet)

    for result in results:
        similares.update(result)

    return list(similares)

In [19]:
agrupado['similaritys'] = agrupado['indices_en_lista'].apply(transformar_lista)

### Asociammos los buckets de tweets, a los usuarios

In [ ]:
def m_caracteristica(subconjun):
    sims = subconjun
    shingles = sims['similaritys'].apply(lambda x: np.array(x))
    mlb = MultiLabelBinarizer(sparse_output=True)
    return mlb.fit_transform(shingles)

In [ ]:
matrix = m_caracteristica(agrupado)

In [ ]:
n_pm= 50
thrld = 0.4
lsh = MinHashLSH(thrld,n_pm)
list_hash = []
for i in range(matrix.shape[0]):
    list_hash.append((i,minhash_vec(matrix[i])))

with lsh.insertion_session() as session:
    for i , minhash in list_hash:
        session.insert(i, minhash)

In [ ]:
id_usuario = 0
m = minhash_vec(matrix[id_usuario])
usuarios_similares = lsh.query(m)